In [12]:
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.utils.data as Data

In [13]:
# Data load
mat1 = sio.loadmat('Bearing/N_0.mat')
data1 = mat1['X097_DE_time']
mat2 = sio.loadmat('Bearing/IR007_0.mat')
data2 = mat2['X105_DE_time']
mat3 = sio.loadmat('Bearing/OR007@6_0.mat')
data3 = mat3['X130_DE_time']
mat4 = sio.loadmat('Bearing/B007_0.mat')
data4 = mat4['X118_DE_time']

Normal = data1[:50000].reshape(100,500) #for individual category, create 100 samples,individual sample contains 500 data points
Fault1 = data2[:50000].reshape(100,500)
Fault2 = data3[:50000].reshape(100,500)
Fault3 = data4[:50000].reshape(100,500)

In [14]:
# prepare training and test data
Normal_tensor=torch.from_numpy(Normal)
Fault1_tensor=torch.from_numpy(Fault1)
Fault2_tensor=torch.from_numpy(Fault2)
Fault3_tensor=torch.from_numpy(Fault3)
Train_Data=torch.cat((Normal_tensor[0:70],Fault1_tensor[:70],Fault2_tensor[:70],Fault3_tensor[:70])).type(torch.FloatTensor)
Train_Data=torch.unsqueeze(Train_Data,1) #unsqueeze convert data from (280,500) to (280,1,500), add additional dimension to match the requirment of data structure in CNN
Test_Data=torch.cat((Normal_tensor[70:],Fault1_tensor[70:],Fault2_tensor[70:],Fault3_tensor[70:])).type(torch.FloatTensor)
Test_Data=torch.unsqueeze(Test_Data,1)
Train_Target=torch.cat((torch.zeros(70,1),torch.ones(70,1),2*torch.ones(70,1),3*torch.ones(70,1)),0).type(torch.LongTensor).squeeze() #squeeze data to create one-dimensional label
Test_Target=torch.cat((torch.zeros(30,1),torch.ones(30,1), 2*torch.ones(30,1),3*torch.ones(30,1)),0).type(torch.LongTensor).squeeze()

In [15]:
# Batch
Train_Dataset=Data.TensorDataset(Train_Data,Train_Target)
Test_Dataset=Data.TensorDataset(Test_Data,Test_Target)
train_loader = Data.DataLoader(
    dataset=Train_Dataset,
    batch_size=10,
    shuffle=True)

In [16]:
# Define 1-D CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (10,1,500) second dimension corresponds to the number of feature maps, third dimension corresponds to the lenght/size of an feature map
            nn.Conv1d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=3,                   
                padding=0,                 # (500-5)/3+1 =166
            ),                              
            nn.ReLU(),                      
            nn.MaxPool1d(kernel_size=3, stride = 2),    # (166-3)/2+1 = 82.5 -> 82
        )
        self.conv2 = nn.Sequential(         # input shape (10, 16, 82)
            nn.Conv1d(16, 32, 5, 2, 0),     # (82-5)/2+1 = 39.5->39
            nn.ReLU(),                      
            nn.MaxPool1d(kernel_size=2, stride = 2)  # (39-2)/2+1 =19.5 -> 19 output shape (10, 32, 19)
        )
        self.out = nn.Linear(608, 4)   # 32*19 = 608 fully connected layer, output 10 classes
        #self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 19)
        x = self.out(x)
        #output = self.softmax(x)
        return output, x    # return x for visualization

cnn = CNN()
print (cnn)

CNN(
  (conv1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(3,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(5,), stride=(2,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=608, out_features=4, bias=True)
)


In [17]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

In [18]:
# Training and Testing
EPOCH = 50
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):   # gives batch data, normalize x when iterate train_loader

        output = cnn(b_x)[1]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        
    if epoch % 2 == 0:
        test_output, last_layer = cnn(Test_Data) #test_output: 60*2
        pred_y = torch.max(last_layer, 1)[1].data.squeeze() #squeeze(): 60*1->(60,)
        accuracy = (pred_y == Test_Target).sum().item() / float(Test_Target.size(0))
        print('Epoch: ', epoch, '| train loss: %.4f' % loss.data, '| test accuracy: %.2f' % accuracy)


Epoch:  0 | train loss: 0.0398 | test accuracy: 0.99
Epoch:  2 | train loss: 0.0007 | test accuracy: 1.00
Epoch:  4 | train loss: 0.0004 | test accuracy: 1.00
Epoch:  6 | train loss: 0.0002 | test accuracy: 1.00
Epoch:  8 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  10 | train loss: 0.0002 | test accuracy: 1.00
Epoch:  12 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  14 | train loss: 0.0002 | test accuracy: 1.00
Epoch:  16 | train loss: 0.0000 | test accuracy: 1.00
Epoch:  18 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  20 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  22 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  24 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  26 | train loss: 0.0000 | test accuracy: 1.00
Epoch:  28 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  30 | train loss: 0.0000 | test accuracy: 1.00
Epoch:  32 | train loss: 0.0001 | test accuracy: 1.00
Epoch:  34 | train loss: 0.0000 | test accuracy: 1.00
Epoch:  36 | train loss: 0.0000 |

In [19]:
output

tensor([[-181.3687,   33.8172,   48.1283,  -64.4762],
        [  -8.0180,   -3.0883,  -23.6354,    7.0944],
        [ -49.4283,    4.7052,   -8.2944,   -8.0258],
        [ -56.7409,    5.0680,   -5.0226,  -11.7922],
        [ -46.6547,    3.5279,  -10.9500,   -7.1388],
        [-190.2363,   33.9392,   51.2618,  -54.0032],
        [  18.1522,   -8.3522,  -30.2409,    6.9355],
        [ -53.8587,    6.4425,   -6.0774,   -9.7510],
        [-164.2679,   28.0877,   44.3358,  -53.4263],
        [ -52.7827,    5.2975,   -6.0121,  -10.6392]],
       grad_fn=<AddmmBackward0>)

In [20]:
import torch
2*torch.ones(100,1).type(torch.LongTensor).squeeze()

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2])